# Inspecting what the LSTM Tweet generator has learned

We trained the LSTM model (A 256 unit LSTM straight into a softmax output layer) on $2^{18}$ (262,144) Tweets. That might not seem like many, but remember each tweet needs 32 training examples (with a windowsize of 64 and a max length of 160 characters) so that's about 8.3 million training examples. Running on an AWS p3.2xlarge instance (on a Tesla V100 GPU) it took about 20 minutes per epoch.

The training took part in two stages (not planned, this was just how it turned out) - first it was trained only on the first half of that data for about 30 epochs. After it looked like the model was starting to over-fit (but without looking like it had got the hang of generating tweets) so I doubled the size of the training set, and ran it for another 10-12 epochs or so.

I say 'looking like' - throughout the training process, I had the model generate a few tweets based on some seed text (randomly pulled in from a tweet) at various settings of the temperature parameter, and eyeballed the resulting gibberish to get a feel for how far through training the model was.

I later realised that this wasn't a particularly good check on the model's ability to generate coherent English language tweets - it seems as though the model is particularly sensitive to what is in the seed text and how much of it it gets. So generating based on 10 or 20 seed characters gets very different results to generating based on 64 seed characters. In an ideal world, we do want the model to be able to freely generate coherent tweets based on a minimal seed (ideally, just an emoji). However, in this notebook we'll take a more detailed look at what's going on.

Let's start by loading up the Twitter dataset that we trained the model on, and the global variables that we used to set the size of the sets.

In [1]:
import numpy as np
import pandas as pd
import data_load_utils as util
from math import ceil

In [2]:
tweets = util.filter_tweets_min_count(util.read_tweet_data('data/emojis_homemade.csv'), min_count=1000)
tweets['text'] = util.filter_text_for_handles(tweets['text'])

MAX_TWEET_LENGTH = 160
WINDOW_SIZE = 64
STEP = 3

samples_per_tweet = int(ceil((MAX_TWEET_LENGTH - WINDOW_SIZE) / STEP)) # 32
tweets_per_batch = 64
samples_per_batch = samples_per_tweet * tweets_per_batch # 2048

chars_univ, chars_univ_idx = util.get_universal_chars_list()

In [3]:
# Here we're going to look at examples from the Dev set but we might want to run analyses on the Train set 

TRAIN_SIZE = 2**18 # 262,144 training examples
DEV_SIZE = 2**12   # 4096 Dev examples

n_train_batches = TRAIN_SIZE / tweets_per_batch
n_dev_batches = DEV_SIZE / tweets_per_batch

tweets_train = tweets.iloc[0:TRAIN_SIZE] 
tweets_dev = tweets.iloc[TRAIN_SIZE:TRAIN_SIZE+DEV_SIZE] 

In [4]:
import keras
from keras import layers
from keras.models import Sequential
from keras import callbacks

Using TensorFlow backend.


In [5]:
model = keras.models.load_model("models/text_emoji_joint_gen_model_man-0.851.hdf5") # 256 LSTM units, ~30 epochs training 

# Examples of generating tweets

In [6]:
def sample (preds, temperature = 1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [7]:
import tensorflow as tf
import random
import sys

In [8]:
def generate_tweet(seed_text, window_size=WINDOW_SIZE,
		   length=MAX_TWEET_LENGTH, temperatures=[0.3, 0.5, 0.8, 1.0],
		   random_seed = None):

    if random_seed:
        np.random.seed(random_seed) # np.random.seed needed by tf 
        tf.set_random_seed(random_seed)
        
    n_seed_chars = window_size
    #first, pad out the seed_text with whitespace     
    seed_text = util.pad_text(seed_text, window_size)

    print ('--- Generating with seed: "' + seed_text + '" [' + str(len(seed_text)) + ']')

    # try a range of sampling temperatures
    for temperature in temperatures:
        generated_text = seed_text
        print ('--------- temperature:', temperature)
        sys.stdout.write(generated_text)

        for i in range (length - n_seed_chars):
            # one-hot encode the characters generated so far
            sampled = np.zeros((1, WINDOW_SIZE, len(chars_univ)))
            for t, char in enumerate (generated_text):
                sampled[0, t, chars_univ_idx[char]] = 1

            # sample the next character
            preds = model.predict(sampled, verbose=0)[0]
            next_index = sample(preds, temperature)
            next_char = chars_univ[next_index]

            generated_text += next_char
            generated_text = generated_text[1:]

            sys.stdout.write(next_char)

        print ("\n")    


In [9]:
def get_seed_text(tweets, seed_length=64, random_seed = None):
    if random_seed:
        np.random.seed(random_seed)
        
    seed_tweet = tweets.iloc[np.random.randint(0, len(tweets))]
    seed_text = seed_tweet['text'][0:seed_length]

    print (seed_text + "\n" + seed_tweet['text'])
    
    return seed_text

In [10]:
seed_text = get_seed_text(tweets, seed_length=0)


I love this!  #Friz https://t.co/T2WiSSvhSt


In [11]:
generate_tweet("", temperatures=[0.4, 0.8, 1.0, 1.2, 1.4])

--- Generating with seed: "                                                                " [64]
--------- temperature: 0.4
                                                                

ValueError: Error when checking model input: the list of Numpy arrays that you are passing to your model is not the size the model expected. Expected to see 2 array(s), but instead got the following list of 1 arrays: [array([[[1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        ...,
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., ...

edit: gibberish when seed_text < 64 chars now fixed. It's still a bit stream-of-consciousness but genuinely I think it's because the model doesn't have much to work from here.

In [25]:
seed_text = get_seed_text(tweets, random_seed=2)
generate_tweet(seed_text, temperatures=[0.4, 0.6, 0.8, 1.0, 1.2, 1.5], random_seed = 2)

RT It doesnt matter how many times Ive watched a vine. Theyre al
RT It doesnt matter how many times Ive watched a vine. Theyre always funny https://t.co/DrWwqvkITn
--- Generating with seed: "RT It doesnt matter how many times Ive watched a vine. Theyre al" [64]
--------- temperature: 0.4
RT It doesnt matter how many times Ive watched a vine. Theyre all and pregand                                                                                   

--------- temperature: 0.6
RT It doesnt matter how many times Ive watched a vine. Theyre all of you  # #United #Pussy #LakeYou #Sarkar #EXO #EXO_TEMPO_DENNO ##KONNEWE_YUE_______EEONTE #BO

--------- temperature: 0.8
RT It doesnt matter how many times Ive watched a vine. Theyre all of the bestmar inspire, sa congration on from #OberSartion! Peace to open out the real team th

--------- temperature: 1.0
RT It doesnt matter how many times Ive watched a vine. Theyre all of my   Thousive Game Of curry this and shat all tralliang video heartan frien

In [26]:
seed_text = get_seed_text(tweets, random_seed=3)
generate_tweet(seed_text, temperatures=[0.4, 0.6, 0.8, 1.0, 1.2, 1.5], random_seed = 3)

Will stick them on the shopping cart and figure out how to use t
Will stick them on the shopping cart and figure out how to use them when I need to 
--- Generating with seed: "Will stick them on the shopping cart and figure out how to use t" [64]
--------- temperature: 0.4
Will stick them on the shopping cart and figure out how to use the same person                                                                                  

--------- temperature: 0.6
Will stick them on the shopping cart and figure out how to use the best selfies  #WorldSeriss                                                                   

--------- temperature: 0.8
Will stick them on the shopping cart and figure out how to use the pike being like a band kind just really working out the world to the Ath Texas first #Sathwye

--------- temperature: 1.0
Will stick them on the shopping cart and figure out how to use the poster for you so let's know, it's fucking dancions!  I love this  but alents' reason  I wis

It's interesting that the model's figured out how to use hashtags - the format ('#' character followed by CamelNotationCaps). Firstly, how they syntactically fit into a tweet, either closing it out a tweet or following a sentence, as punctuation (e.g., #Sathwye #JifksyjK0 etc) or as a #WordReplacement (e.g., #Shivagsuntar etc). It's also figured out how URLs work in tweets.

It's perhaps unsurprising that hashtags were one of the first things the model got the hang of during training (note to self - flesh this point out with an example of the model earlier during training). They're used consistently across multiple different tweets, often more regularly than meaningful english language words, and given how the data was collected (basically, whenever I remembered to leave the tweet harvesting script running) there's likely to be bands of uniformity across the data. (note to self - could plot a graphic of hashtag use over time throughout the dataset)

In [27]:
seed_text = get_seed_text(tweets, random_seed=4)
generate_tweet(seed_text, temperatures=[0.4, 0.6, 0.8, 1.0, 1.2, 1.5], random_seed = 4)

RT Notch necessities  https://t.co/hQooQ5l7pE
RT Notch necessities  https://t.co/hQooQ5l7pE
--- Generating with seed: "                   RT Notch necessities  https://t.co/hQooQ5l7pE" [64]
--------- temperature: 0.4
                   RT Notch necessities  https://t.co/hQooQ5l7pE      # # #                                                                                     

--------- temperature: 0.6
                   RT Notch necessities  https://t.co/hQooQ5l7pE  Love you#kimpa #hank #sweater #Khoweach # #ALDUBStppeNeaMeNTame # #SimoneYou #Sarkar #Shoothea

--------- temperature: 0.8
                   RT Notch necessities  https://t.co/hQooQ5l7pE https://t.co/YPpSiRemjM #AfterNow #NewAflaePoungyou #EXO #EXO #weareoneEXO #EXO_TEMPO #EXO_Dont

--------- temperature: 1.0
                   RT Notch necessities  https://t.co/hQooQ5l7pE w htt/stpo:ma. linded precious and only EXO-Ls 3 5. #VoteGornime_Young #findomGointende #MaineS

--------- temperature: 1.2
                   RT Notch

In [28]:
seed_text = get_seed_text(tweets, random_seed=5)
generate_tweet(seed_text, temperatures=[0.4, 0.6, 0.8, 1.0, 1.2, 1.5], random_seed = 5)

I don't like nobody that likes me 
I don't like nobody that likes me 
--- Generating with seed: "                              I don't like nobody that likes me " [64]
--------- temperature: 0.4
                              I don't like nobody that likes me a start shit streams to start  the season    #GOT7 # # # # # # # # # # # # # # # #              

--------- temperature: 0.6
                              I don't like nobody that likes me happy this she looks like they are so proud of you to the memorian no hair  #IZONE # #  #       

--------- temperature: 0.8
                              I don't like nobody that likes me and watching me to rubing the first song is a script cus a girls make your special shirt  Get my

--------- temperature: 1.0
                              I don't like nobody that likes me    Just Debrica menos with on a bOULING CROINTDRC Arlance (hmo aint knofrie Phankeequich  i fund

--------- temperature: 1.2
                              I don't like nobody

In [29]:
seed_text = get_seed_text(tweets, random_seed=6)
generate_tweet(seed_text, temperatures=[0.4, 0.6, 0.8, 1.0, 1.2, 1.5], random_seed = 6)

always 
always 
--- Generating with seed: "                                                         always " [64]
--------- temperature: 0.4
                                                         always the most at the brother  #WasteItOnMe  # # # #                                                 :

--------- temperature: 0.6
                                                         always congratulations and sounds man taking every of my Feel looking from that you broke more than lif

--------- temperature: 0.8
                                                         always hang shoes the really new to see the fucking the sure deserves a president and stuff show cumsul

--------- temperature: 1.0
                                                         always there achieve but do a new Japanala and Im so excited again. no legend. Heard some genysonatform

--------- temperature: 1.2
                                                         always wait at LAA 10 seconds  lemin memondy,c

In [30]:
seed_text = get_seed_text(tweets, random_seed=7)
generate_tweet(seed_text, length=300, temperatures=[0.4, 0.6, 0.8, 1.0, 1.2, 1.5], random_seed = 7)

RT Here's my plan:1 - Vote by mail for &amp; 2 - Volunteer to GO
RT Here's my plan:1 - Vote by mail for &amp; 2 - Volunteer to GOTV betwee
--- Generating with seed: "RT Here's my plan:1 - Vote by mail for &amp; 2 - Volunteer to GO" [64]
--------- temperature: 0.4
RT Here's my plan:1 - Vote by mail for &amp; 2 - Volunteer to GOT to #SGA #LightinSunta   #MTVEMABiggestFansBTS #BTS #BTS  # # # # # # # # # # # # # # # #___________E @poojial #support # # # # # # # # # # # # # # # # # #                                                         : dancing the same time

--------- temperature: 0.6
RT Here's my plan:1 - Vote by mail for &amp; 2 - Volunteer to GORLALL TIMULick in Discon extending  #ALDUBAlwaysForLove #cospens #really #iKON #WOREALLY # # #SexyFans #SeetherSasan_DA #Sarkar #Sarkar #TheFanal #MoreHeren #RM #Soribana #TEMPO https://t.co/vcZBWaE65RW#EXO #EXO_TEMPO# #change # # # # #

--------- temperature: 0.8
RT Here's my plan:1 - Vote by mail for &amp; 2 - Volunteer to GO MILION suppor

I like `Volunteer to GOD is way`. But the model doesn't seem to have got that the theme of the seed was political (`Vote by mail for &amp`) and has come up with a load of unrelated hashtags (a spot of Googling reveals that `#LightinSunta'` was invented by the model, although [`#MTVEMABiggestFansBTS` exists, it's a K-pop thing](https://twitter.com/hashtag/mtvemabiggestfansbts). #Sarkar is an [Indian Tamil-language action drama film](https://en.wikipedia.org/wiki/Sarkar_(2018_film).

In [31]:
seed_text = get_seed_text(tweets, random_seed=8)
generate_tweet(seed_text, temperatures=[0.4, 0.6, 0.8, 1.0, 1.2, 1.5], random_seed = 8)

RT  Tom Arthur to Tory Michelle Ballantyne: "To suggest that pov
RT  Tom Arthur to Tory Michelle Ballantyne: "To suggest that poverty should be a barrier to a family, that people who are poor ar
--- Generating with seed: "RT  Tom Arthur to Tory Michelle Ballantyne: "To suggest that pov" [64]
--------- temperature: 0.4
RT  Tom Arthur to Tory Michelle Ballantyne: "To suggest that pove the back and you look at his homee                                                            

--------- temperature: 0.6
RT  Tom Arthur to Tory Michelle Ballantyne: "To suggest that pove, Were the deviler of her second week of the new subred  #Kangashi   #bunnkin #parth #prounda #

--------- temperature: 0.8
RT  Tom Arthur to Tory Michelle Ballantyne: "To suggest that pove be bed to work into people for the body and you havent be so preciots in so lond to that conte

--------- temperature: 1.0
RT  Tom Arthur to Tory Michelle Ballantyne: "To suggest that povesmeding! xill the street  #withlovie #LegendTers

In [32]:
seed_text = get_seed_text(tweets, random_seed=9)
generate_tweet(seed_text, length=300, temperatures=[0.4, 0.6, 0.8, 1.0, 1.2, 1.5], random_seed = 9)

RT Life comes at you fast. One day you own a machine gun...the n
RT Life comes at you fast. One day you own a machine gun...the next you're on the floor, handcuffed, with your face betw
--- Generating with seed: "RT Life comes at you fast. One day you own a machine gun...the n" [64]
--------- temperature: 0.4
RT Life comes at you fast. One day you own a machine gun...the new shit to the time the only one of the same time  #EXO #EXO_TEMPO #EXO_DontMessUpMyTempo #EXO_TEMPO #EXO_DontMessUpMyTempo #EXO_TEMPO #EXO_DontMessUpMyTempo #EXO_TEMPO #EXO_DontMessUpMyTempo #namjun #who #change # # # # # # # # # # # #

--------- temperature: 0.6
RT Life comes at you fast. One day you own a machine gun...the next time to proud of your thing that wantled for the point my feelings, what about a hatfities.   #HeartFreaking #Daytion #SarkarDays #MexicoGP  #SaveShaden #MWatherBight #BTS #Kanahhthariing #1                                          

--------- temperature: 0.8
RT Life comes at you fast. One 

The model really likes K-Pop `#EXO #EXO #EXO_TEMP #EXO_DontMessUpMyTempo`

In [33]:
seed_text = get_seed_text(tweets, random_seed=10)
generate_tweet(seed_text, temperatures=[0.4, 0.6, 0.8, 1.0, 1.2, 1.5], random_seed = 10)

youre so cute 
youre so cute 
--- Generating with seed: "                                                  youre so cute " [64]
--------- temperature: 0.4
                                                  youre so cute                                                                                                 

--------- temperature: 0.6
                                                  youre so cute that beautiful and what a big kinda kids this  I need a people go on the charts  https://t.co/yy

--------- temperature: 0.8
                                                  youre so cute because he got to fort they &amp; youre call me to Chana  Leadnagi Nearing on, about to see this

--------- temperature: 1.0
                                                  youre so cute thinking also fans RT I ever havent people saidan  calls hi turnk over 4 and houry! theyre dress

--------- temperature: 1.2
                                                  youre so cute #mzvicluckibaPie Beanon (

In [34]:
seed_text = get_seed_text(tweets, random_seed=11)
generate_tweet(seed_text, temperatures=[0.4, 0.6, 0.8, 1.0, 1.2, 1.5], random_seed = 11)

RT DJ with Gforce michael.gwapooo https://t.co/U4dVK54RJp https:
RT DJ with Gforce michael.gwapooo https://t.co/U4dVK54RJp https://t.co/ZD5SvE8NZl
--- Generating with seed: "RT DJ with Gforce michael.gwapooo https://t.co/U4dVK54RJp https:" [64]
--------- temperature: 0.4
RT DJ with Gforce michael.gwapooo https://t.co/U4dVK54RJp https://t.co/W0DdeZbVD  # # # # #                                                       :             

--------- temperature: 0.6
RT DJ with Gforce michael.gwapooo https://t.co/U4dVK54RJp https://t.co/8wuvfyLKhg #Bobbles #BTS https://t.co/frdatXiH47 #YETO # #GOT7 #Go #ALDUB1ttpre hears  #T

--------- temperature: 0.8
RT DJ with Gforce michael.gwapooo https://t.co/U4dVK54RJp https://t.co/uMgamlbfId#Genuraz_deviendelun-Taiq  love to says next very after Appending time.  thank 

--------- temperature: 1.0
RT DJ with Gforce michael.gwapooo https://t.co/U4dVK54RJp https://t.co/He3j5knhjy   #FTVRIBAGET DIDITHCA GUTS VOTING SO WANT TO BE CheSolk-at Bobs in being-Coll


In [35]:
seed_text = get_seed_text(tweets, random_seed=12)
generate_tweet(seed_text, temperatures=[0.4, 0.6, 0.8, 1.0, 1.2, 1.5], random_seed = 12)

Because your definitely sleeping  https://t.co/5kUsa6t9do
Because your definitely sleeping  https://t.co/5kUsa6t9do
--- Generating with seed: "       Because your definitely sleeping  https://t.co/5kUsa6t9do" [64]
--------- temperature: 0.4
       Because your definitely sleeping  https://t.co/5kUsa6t9do #First #ALDUBTotally  lot of hour the beather and stay the song and support a confest play in a

--------- temperature: 0.6
       Because your definitely sleeping  https://t.co/5kUsa6t9do  https://t.co/tEILsnTrwd   #bbocks #meal #give #sexy #pics #laving #shadom #drivies shift and s

--------- temperature: 0.8
       Because your definitely sleeping  https://t.co/5kUsa6t9do  the hell that repeal that the first him on a chark this and a follow me                       

--------- temperature: 1.0
       Because your definitely sleeping  https://t.co/5kUsa6t9do Heact-Lare and The commen  VIncable pilks  @kpeckeych #baboy  https://t.co/0ZdDm73Y77 #StreamWa

--------- temperature: 1.2
   

In [88]:
seed_text = get_seed_text(tweets, random_seed=13)
generate_tweet(seed_text, temperatures=[0.4, 0.6, 0.8, 1.0, 1.2, 1.5], random_seed = 13)

RT namjoon: &lt;3  &lt;3jimin: *is absolutely devastated* https:
RT namjoon: &lt;3  &lt;3jimin: *is absolutely devastated* https://t.co/dI3s2sh3MN
--- Generating with seed: "RT namjoon: &lt;3  &lt;3jimin: *is absolutely devastated* https:"
--------- temperature: 0.4
RT namjoon: &lt;3  &lt;3jimin: *is absolutely devastated* https://t.co/hddW4lw6ey  https://t.co/HJLLaasbB  #Gottanding #BTS #MAMAVOTE #BTS #BTS #BTS #BTS       

--------- temperature: 0.6
RT namjoon: &lt;3  &lt;3jimin: *is absolutely devastated* https://t.co/yucjb4yjht #Allance #Sunewi We can go if it is too extra for an easy she Savi Sampical wi

--------- temperature: 0.8
RT namjoon: &lt;3  &lt;3jimin: *is absolutely devastated* https://t.co/hfdWUYbvua  #100 #KingleTurneTurn # So Ill Kade Rama is what an everyday girl has been su

--------- temperature: 1.0
RT namjoon: &lt;3  &lt;3jimin: *is absolutely devastated* https://t.co/bux4KoBIEV more  #Goodgeal compleaving and at the Eitch Compans and  those women BigHing 

----

In [106]:
seed_text = get_seed_text(tweets, random_seed=24)
generate_tweet(seed_text, temperatures=[0.4, 0.6, 0.8, 1.0, 1.2, 1.5], random_seed = 24)

RT Ariana will be releasing the title track of her upcoming albu
RT Ariana will be releasing the title track of her upcoming album Thank U, Next tonight! #ThankUNext  https://t.co/sPLAhiH
--- Generating with seed: "RT Ariana will be releasing the title track of her upcoming albu"
--------- temperature: 0.4
RT Ariana will be releasing the title track of her upcoming albums and send my man  https://t.co/1B0hNvrHvi                                                     

--------- temperature: 0.6
RT Ariana will be releasing the title track of her upcoming album  shes looks so anyone cut the time to an advice  https://t.co/hLuG7udMex          #Got #LikePl

--------- temperature: 0.8
RT Ariana will be releasing the title track of her upcoming album  I'm truth they are they get #MAMAVOPENT me  The masket dress under me  https://t.co/BrOyxdl3K

--------- temperature: 1.0
RT Ariana will be releasing the title track of her upcoming album is partnez. They are everyone and disajarl for the Nikkday 

In [111]:
seed_text = get_seed_text(tweets, random_seed=28)
generate_tweet(seed_text, temperatures=[0.4, 0.6, 0.8, 1.0, 1.2, 1.5], random_seed = 28)

Titles never matter ... The willing will always do what they wan
Titles never matter ... The willing will always do what they want ... 
--- Generating with seed: "Titles never matter ... The willing will always do what they wan"
--------- temperature: 0.4
Titles never matter ... The willing will always do what they want to college the and of my reast are the only one is already surprised to the next time in an al

--------- temperature: 0.6
Titles never matter ... The willing will always do what they want to see the same to big drinks and never access  and your favorite sungely                     

--------- temperature: 0.8
Titles never matter ... The willing will always do what they want  SJA in sports                                                                                

--------- temperature: 1.0
Titles never matter ... The willing will always do what they wanna skyli I want to see 4 guing making me and shit her growing my headed about you, I putated bui

--------- tempe

In [113]:
seed_text = get_seed_text(tweets, random_seed=30)
generate_tweet(seed_text, temperatures=[0.4, 0.6, 0.8, 1.0, 1.2, 1.5], random_seed = 30)

Make your opponent pay for your suffering in the holidays girl !
Make your opponent pay for your suffering in the holidays girl !!!! 
--- Generating with seed: "Make your opponent pay for your suffering in the holidays girl !"
--------- temperature: 0.4
Make your opponent pay for your suffering in the holidays girl !!                                                                                               

--------- temperature: 0.6
Make your opponent pay for your suffering in the holidays girl !! I was just have to do the sexy baby on the best                                               

--------- temperature: 0.8
Make your opponent pay for your suffering in the holidays girl !                                                                                               R

--------- temperature: 1.0
Make your opponent pay for your suffering in the holidays girl !  Every Meson             , pass  whats but what I need to playing it  nope the "Heles_EP crock,

--------- tempera

In [121]:
seed_text = "Someone took the dumbbells I left outside the house, #Strong #Go"
print (len(seed_text))
generate_tweet(seed_text, temperatures=[0.4, 0.6, 0.8, 1.0, 1.2, 1.5], random_seed = 1)

64
--- Generating with seed: "Someone took the dumbbells I left outside the house, #Strong #Go"
--------- temperature: 0.4
Someone took the dumbbells I left outside the house, #Strong #Go #Authand #Good #Fillow #TheFirst # # #  # #   #Jimin #MTVEMABiggestFansBTS #BTS #MTVEMABiggestF

--------- temperature: 0.6
Someone took the dumbbells I left outside the house, #Strong #Go #DGFTinteress #Chue #Castary # # # # #                                                    :: yo

--------- temperature: 0.8
Someone took the dumbbells I left outside the house, #Strong #God #Nexx #valuman # # # #           @MyPJ 11923 3 3 2Parandom - 1008-187  Spinniestheh https://t.

--------- temperature: 1.0
Someone took the dumbbells I left outside the house, #Strong #GoDasket #Fansanay Watco - I watch 10.2bCos out if your skate with sister looking amazing  honest 

--------- temperature: 1.2
Someone took the dumbbells I left outside the house, #Strong #Go Day?Itely Vija FHY HE LOOK AND MEAS HR DONIXIL MION BABA

## Looking at distribution of the training set - what proportion are preceding whitespace?
ie. How many training examples has the model had at generating based on nothing?

NameError: name 'date' is not defined